In [1]:
# !python -m pip install mlagents==0.28.0
# !python -m pip install gym
# !cd "/Users/aditya/Documents/GitHub/game_creation_research/ml-agents/gym-unity" && pip3 install -e .

from PlaceAndShootGym import *
from mlagents_envs.environment import UnityEnvironment
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel


In [2]:
# GAME1 = Shoot into bucket through many different gaps for variable rewards
# GAME2 = Shoot into bucket after colliding with crate once
# GAME3 = Shoot and land onto a platform on the top right made of corner and crate
# GAME4 = shoot and touch as many objects before it touches the floor and playable if more than 6 points
# GAME5 = free the ball and balance on gear until it goes into bucket (challenge scenario)

In [36]:
def GAME_1_REWARD(obsVec: List[Obs]) -> int:
    if not endsInBucket(obsVec):
        return 0
    def top(pos, triangle):
        return pos.y>triangle.y
    def mid1(pos, corner, triangle):
        return pos.y>corner.y and pos.y<triangle.y
    def mid2(pos, crate, corner):
        return pos.y>crate.y and pos.y<corner.y
    def bottom(pos, crate):
        return pos.y<crate.y
    for each_obs in obsVec:
        pos = each_obs.ballPos
        if bottom(pos, each_obs.objPos["crate"]): return 4
        if mid2(pos, each_obs.objPos["crate"], each_obs.objPos["corner"]): return 3
        if mid1(pos, each_obs.objPos["corner"], each_obs.objPos["triangle"]): return 2
        if top(pos, each_obs.objPos["triangle"]): return 1
    return 0

In [37]:
GAME_1_TRANSFORMER = copy.deepcopy(NO_OBJECT_INTERACTION)
GAME_1_TRANSFORMER.ban_mouse_position_x = (-1, -0.5)
GAME_1_TRANSFORMER

{"ban_object": ["crate", "bucket", "corner", "gear", "triangle"], "ban_mouse_position_x": [-1, -0.5], "ban_mouse_position_y": [99, 999], "ban_object_position_x": [99, 999], "ban_object_position_y": [99, 999], "default_action": [-0.001, 0, 0, 0, 0, 0]}

In [38]:
try:
    env.close()
except:
    pass

SERVER_BUILD = "../Builds/MLAgent_View_21April22_server.app"
GYM_BUILD = "../Builds/Gym_View_25April22.app"

# channel = EngineConfigurationChannel()
# channel.set_configuration_parameters(time_scale=5, quality_level=0)
# unity_env = UnityEnvironment(
#     file_name=GYM_BUILD, seed=1, side_channels=[channel], worker_id=0)

unity_env = UnityEnvironment()

# Start interacting with the environment.
unity_env.reset()
gym_env = UnityToGymWrapper(unity_env, allow_multiple_obs=False)
env = PlaceAndShootGym(gym_env, reward_fn=GAME_1_REWARD,
                       actionTransformer=GAME_1_TRANSFORMER,
                       announce_actions=True)


[INFO] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
[INFO] Connected to Unity environment with package version 2.2.1-exp.1 and communication version 1.5.0
[INFO] Connected new brain: PlaceAndShoot?team=0
[WARNING] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.


/opt/anaconda3/envs/introml_conda/lib/python3.8/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


In [39]:
GAME_1_SETUP = [[0, 0, -0.55, 0.6, Action.objectTagToActionVal("triangle"), 0],
                [0, 0, -0.55, 0.1, Action.objectTagToActionVal("corner"), 0],
                [0, 0, -0.55, -0.4, Action.objectTagToActionVal("crate"), 0],
                [0, 0, -0.85, -0.91, Action.objectTagToActionVal("bucket"), 0],
                [0, 0, 0, 0, 0, 1]]

env.setup(GAME_1_SETUP)

In [7]:
# reset pedestal

a = Action()
a.reset = True
print(Obs(env.step(a)[0]))


[0, 0, 0, 0, None, True]
bucket: Vector2(x=-3.825, y=-4.0950003)
corner: Vector2(x=-2.4750001, y=0.45000002)
crate: Vector2(x=-2.4750001, y=-1.8000001)
gear: Vector2(x=6.75, y=-0.65)
triangle: Vector2(x=-2.4750001, y=2.7)
Ball Position: Vector2(x=0.0, y=-3.9)
Ball Velocity: Vector2(x=0.0, y=-0.0)
Collided With: None
In Reset?: True



In [8]:
# see exactly what just happened

for each_obs in env.lastObsVec:
    print(each_obs)


bucket: Vector2(x=-3.825, y=-4.0950003)
corner: Vector2(x=-2.4750001, y=0.45000002)
crate: Vector2(x=-2.4750001, y=-1.8000001)
gear: Vector2(x=6.75, y=-0.65)
triangle: Vector2(x=-2.4750001, y=2.7)
Ball Position: Vector2(x=0.0, y=-3.9)
Ball Velocity: Vector2(x=0.0, y=-0.0)
Collided With: None
In Reset?: True



In [44]:
# manual actioner

action = Action()
action.mouseX = 0.4
action.mouseY = -0.5
action.objX = 0
action.objY = 0
# action.setObject("crate")
# action.reset = True
env.step(action)


[0.4, -0.5, 0, 0, None, False]


(array([-3.8250000e+00, -4.0950003e+00, -2.4750001e+00,  4.5000002e-01,
        -2.4750001e+00, -1.8000001e+00,  6.7500000e+00, -6.4999998e-01,
        -2.4750001e+00,  2.7000000e+00, -3.8911114e+00, -4.2995667e+00,
         1.1591169e-09, -3.5963228e-09,  1.0000000e+00,  0.0000000e+00],
       dtype=float32),
 3.0,
 False,
 {'step': <mlagents_envs.base_env.DecisionSteps at 0x7fb4f7068910>})

In [10]:
# env.close()


In [11]:
# env.reset()
